# Lab07 - Sparsity Aware Learning
Author: [Yunting Chiu](https://www.linkedin.com/in/yuntingchiu/) adapted from Dr. Zois Boukouvalas


# Install the required packages

In [2]:
import matplotlib.pyplot as plt
import math
import numpy as np
import sys
import os

# Exercise 1

For this exercise, the performance of the SVM is tested in the context of a two-class two- dimensional classification task. The data set comprises $N$ = 150 points uniformly distributed in the region $[−5, 5] \times [−5, 5]$. For each point $x_n = [x_n,1, x_n,2]^T$, we compute 

<br/> 
$$
y_n = 0.05x^3_{n,1} + 0.05x^2_{n,1} + 0.05x_{n,1} + 0.05 + \eta,
$$
<br/>
where $\eta$ stands for zero mean Gaussian noise of variance $\sigma^2_{\eta}$ = 4. The point is assigned to either of the two classes, depending on the value of the noise as well as its position with respect to the graph of the function

<br/>
$$
f (x) = 0.05x^3 + 0.05x^2 + 0.05x + 0.05
$$

in the two-dimensional space. That is, if $x_{n,2} \geq y_n$, the point is assigned to class $w1$; otherwise, it is assigned to class $w2$.

In [ ]:
def kappa(x, y, kernel_type, kernel_params): value = None
  if kernel_type == 'gaus':
    sigma = kernel_params[0]
    N = x.shape[0]
    norm = np.sum((x-y)**2)
    value = np.exp(-norm/(sigma**2))
  elif kernel_type == 'gaus_c':
    sigma = kernel_params[0]
    N = len(x)
    exponent = sum( (x-y.conj())**2 )
    # value = 2*(math.exp( -np.real(exponent)/(sigma**2) )) 
    value = 2*np.real(np.exp(-exponent/(sigma**2)))
  elif kernel_type == 'linear': 
    value = 0
    N = len(x)
    for i in range(0, N):
      value = value + x[i]*y[i].conj() 
  elif kernel_type == 'poly':
    d = kernel_params[0]
    value = (1 + np.dot(x, y.conj().transpose()))**d
  # value = ( (1 + x*y.transpose())/( math.sqrt(np.real(x*x.transpose())* np.real(y*y.transpose()) ) ) )**d; 
  elif kernel_type == 'poly_c':
    d = kernel_params[0]
    value = 2*np.real((1 + np.dot(x, y.conj().transpose()))**d)
  # value = 2*np.real( ( (1 + x*y.transpose())/( math.sqrt(np.real(x*x.transpose()) * np.real(y*y.transpose()) ) ) )**d ) 
  return value

In [ ]:
global a1_g, a2_g, b_g, u_g, KKT_g, NB_g, a1i_new, a1j_new, a2i_new, a2j_new
"""
# SMO algorithm for classification.
# The Algorithm computes the parameters a_k, of the expansion of the solution w = Sum a_n*K(. , x_n) and the parameter b.
# -------------------------------------------------------------------
# input variables
# x:
# y:
# C:
# -----------------------------------------------------------------
# output variables
# a  :
# b  :
"""
global a1_g, a2_g, b_g, u_g, KKT_g, NB_g, a1i_new, a1j_new, a2i_new, a2j_new

def SMO_classification(x, y, C, epsilon, kernel_type, kernel_params):
  global a_g, b_g, u_g, KKT_g, NB_g tol = 0.001
  [M, N] = x.shape
  Kernel_matrix = np.zeros(shape=(M, M))
  if kernel_type == 'gaus':
  par = kernel_params
  norms = np.zeros(shape=(M, M)) for i in range(0, M):
  T = x - x[i, :] # bsxfun(@minus,x,x(i,:)) norms[i, :] = np.sum(T ** 2, axis=1)
  Kernel_matrix[:, :] = np.exp(-norms / (par ** 2)) else:
  for i in range(0, M): for j in range(0, M):
  a M x N matrix. It contains the input vectors x_i (each one in
  every row.
  a vector of size M. The class of each input vector
  the trade-off parameter of the SVM model
  a vector of size m. These are the support vectors
  a real number. This is the offset of the support vector
  expansion.
  2

  Kernel_matrix[i, j] = kappa(x[i, :], x[j, :], kernel_type,␣ 􏰣→kernel_params)
  # initialize the support vectors
  a_g = np.zeros(shape=(M, )) #initialize the threshold
  b_g = 0
  #u contains the values of the sv expansion for each input
  u_g = np.zeros(shape=(M, ))
  # KKT(i) is 1 if the i-th sv (a(i)) satisfies the KKT conditions # KKT(i) is 0 otherwise
  KKT_g=np.zeros(shape=(M, ))
  # Update KKT conditions
  for k in range(0, M):
  r2 = (u_g[k]-y[k])*y[k]
  if ((r2 < -tol) and (a_g[k] < C)) or ((r2 > tol) and (a_g[k] > 0) ):
  KKT_g[k] = 0 # KKT condition not satisfied else:
  KKT_g[k] = 1
  # NB(i) is 1 if the sv a(i) is non bound, i.e. 0<a(i)<C # NB(i) is 0 otherwise.
  NB_g = np.zeros(shape=(M, ))
  numChanged=0
  examineAll=1
  while (numChanged > 0) or (examineAll == 1):
  numChanged=0
  if examineAll==1:
  # loop over all training examples
  for i in range(0, M):
  numChanged = numChanged + examineExample(i, x, y, C, epsilon,␣
  􏰣→Kernel_matrix)
  else:
  # loop over all training examples, where a(i) is non-bound for i in range(0, M):
  if (NB_g[i] == 1):
  numChanged = numChanged + examineExample(i, x, y, C,␣
  􏰣→epsilon, Kernel_matrix)
  if examineAll == 1: examineAll = 0
  3

  elif numChanged == 0: examineAll = 1
  a = np.array(a_g)
  b = np.array(b_g)
  print('SMO Finished')
  return a, b

(a) Plot the points $[x_{n,1},x_{n,2}]$ using different colors for each class.

(b) Use SVM with the Gaussian kernel for $\sigma$ = 20 and set $C$ = 1. Plot the classifier and the margin. Moreover, find the support vectors (i.e., the points with nonzero Lagrange multipliers that contribute to the expansion of the classifier) and plot them as circled points.

(c) Repeat step (b) using $C$ = 0.5, 0.1, 0.05.

(d) Repeat step (b) using $C$ = 5, 10, 50, 100.

(e) Comment on the results.

# Output

In [ ]:
# should access the Google Drive files before running the chunk
%%capture
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic 
!jupyter nbconvert --to pdf "/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/GitHub/Labs/07/submit/Lab7_Yunting.ipynb"